In [57]:
import sys
sys.executable

'/opt/anaconda3/envs/poker/bin/python'

In [36]:
import tensorflow as tf

# How to use this

Run each cell from top to bottom. 
View README.md for more infos. 

In [7]:
# Init global infos
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, Activation

from tensorflow.keras.optimizers import SGD

from tensorflow.keras.utils import to_categorical

tf.keras.backend.clear_session()


In [8]:
pokerdf = pd.read_csv('train.csv',index_col = False)

In [9]:
pokercols = pokerdf.columns

In [10]:
#Defining a new input for my Adam Poker application
inputs = ( 
    ("S1", ('1','2','3','4')), 
    ("C1", ("continuous",)), 
    ("S2", ('1','2','3','4')), 
    ("C2", ("continuous",)), 
    ("S3", ('1','2','3','4')), 
    ("C3", ("continuous",)), 
    ("S4", ('1','2','3','4')), 
    ("C4", ("continuous",)), 
    ("S5", ('1','2','3','4')), 
    ("C5", ("continuous",)), 
)

In [11]:
input_shape = []
for i in inputs:
    count = len(i[1 ])
    input_shape.append(count)
input_dim = sum(input_shape)
print("input_shape:", input_shape)
print("input_dim:", input_dim)

input_shape: [4, 1, 4, 1, 4, 1, 4, 1, 4, 1]
input_dim: 25


In [12]:
outputs = tuple(range(0,10))  # (">50K", "<=50K")
output_dim = len(outputs)
print("output_dim:", output_dim)
print()

output_dim: 10



In [13]:
# Helpful function 2    
    
def find_means_for_continuous_types(X):
    means = []
    for col in range(len(X[0])):
        summ = 0
        count = 0.000000000000000000001
        for value in X[:, col]:
            if isinstance(value,float): 
                summ += value
                count +=1
        means.append(summ/count)
    return means

## Changing dtype to float

In [14]:
pokerdf[['C1','C2','C3','C4','C5']] = pokerdf[['C1','C2','C3','C4','C5']].astype('float64')
pokerdf[['S1','S2','S3','S4','S5']] = pokerdf[['S1','S2','S3','S4','S5']].astype('str')

In [15]:
# Helpful function 3
def prepare_data(raw_data, means):
    
    X = raw_data[:, :-1]
    y = raw_data[:, -1:]
    
    # X:
    def flatten_persons_inputs_for_model(person_inputs):
        global inputs
        global input_shape
        global input_dim
        global means
        float_inputs = []

        for i in range(len(input_shape)):
            features_of_this_type = input_shape[i]
            is_feature_continuous = features_of_this_type == 1

            if is_feature_continuous:
                mean = means[i]
                scale_factor = 1/(2*mean)
                float_inputs.append(person_inputs[i]*scale_factor)
#                 if isinstance(person_inputs[i],float):
#                     scale_factor = 1/(2*mean)  # we prefer inputs mainly scaled from -1 to 1. 
#                     float_inputs.append(float(person_inputs[i])*scale_factor)
#                 else:
#                     float_inputs.append(mean)
            else:
                for j in range(features_of_this_type):
                    feature_name = inputs[i][1][j]

                    if feature_name == person_inputs[i]:
                        float_inputs.append(1.)
                    else:
                        float_inputs.append(0)
        return float_inputs
    
    new_X = []
    for person in range(len(X)):
        formatted_X = flatten_persons_inputs_for_model(X[person])
        new_X.append(formatted_X)
    new_X = np.array(new_X)
    
    # y:
    new_y = to_categorical(y, num_classes = 10)
    
#     new_y = []
#     for i in range(len(y)):
#         if y[i] == ">50k":
#             new_y.append((1, 0))
#         else:  # y[i] == "<=50k":
#             new_y.append((0, 1))
#     new_y = np.array(new_y)
    
    return (new_X, new_y)

## Finding means for poker data below

In [16]:
hartrain = pokerdf.values

In [17]:
means = find_means_for_continuous_types(hartrain)
print("Mean values for data types (if continuous):", means)

Mean values for data types (if continuous): [0.0, 6.995241903238704, 0.0, 7.014194322271091, 0.0, 7.014154338264694, 0.0, 6.9424630147940825, 0.0, 6.962734906037585, 0.0]


In [18]:
X_train, y_train = prepare_data(hartrain, means)

In [19]:
# Explanation on data format
print("Training data format example:")
print(X_train[4])  # 4 is a random person, from cuba.

Training data format example:
[1.         0.         0.         0.         0.57181725 0.
 1.         0.         0.         0.2851361  0.         1.
 0.         0.         0.78412874 0.         1.         0.
 0.         0.1440411  0.         1.         0.         0.
 0.07181086]


In [52]:
# Init model

mid_dim = 2000

sec_dim = 1000

model = Sequential()

model.add(Dense(mid_dim, input_dim=input_dim, activation='relu'))
model.add(Dense(sec_dim, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'] )

model.summary()

In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 2000)              52000     
_________________________________________________________________
dense_12 (Dense)             (None, 1000)              2001000   
_________________________________________________________________
dense_13 (Dense)             (None, 10)                10010     
Total params: 2,063,010
Trainable params: 2,063,010
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Train the model

print("(training_datas, dimension):", X_train.shape)

(training_datas, dimension): (25010, 25)


In [55]:
# model.fit(new_X_train, y_train, nb_epoch=3, batch_size=16, show_accuracy=True, verbose=2)
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/5
313/313 [==============================] - 1s 4ms/step - loss: 1.0095 - accuracy: 0.4842 - val_loss: 0.9842 - val_accuracy: 0.4856
Epoch 2/5
313/313 [==============================] - 1s 3ms/step - loss: 0.9713 - accuracy: 0.5024 - val_loss: 0.9665 - val_accuracy: 0.5248
Epoch 3/5
313/313 [==============================] - 1s 3ms/step - loss: 0.9502 - accuracy: 0.5177 - val_loss: 0.9565 - val_accuracy: 0.5282
Epoch 4/5
313/313 [==============================] - 1s 3ms/step - loss: 0.9347 - accuracy: 0.5384 - val_loss: 0.9587 - val_accuracy: 0.5184
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.9272 - accuracy: 0.5405 - val_loss: 0.9562 - val_accuracy: 0.5444


## New stuff here

In [120]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [123]:
from tensorflow.keras.callbacks import LearningRateScheduler

# This is a sample of a scheduler I used in the past
def lr_scheduler(epoch, lr):
    decay_rate = 0.85
    decay_step = 1
    if epoch % decay_step == 0 and epoch:
        return lr * pow(decay_rate, np.floor(epoch / decay_step))
    return lr

In [124]:
callbacks = [LearningRateScheduler(lr_scheduler, verbose=1)]

In [117]:
loss = CategoricalCrossentropy(label_smoothing=0.2)

In [139]:
optim = Adam(learning_rate=0.1)

In [140]:
mid_dim = 100

sec_dim = 50

third_dim = 50

model = Sequential()

model.add(Dense(mid_dim, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(sec_dim, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(third_dim, activation='relu'))
model.add(Dropout(0.02))
model.add(Dense(output_dim, activation='softmax'))

model.compile(loss=loss, optimizer='Adam',metrics = ['accuracy'])

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 100)               2600      
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 10)              

In [142]:
model.fit(X_train, y_train,epochs=30, validation_split=0.2, batch_size=64, verbose=1)

Epoch 1/30
313/313 [==============================] - 1s 3ms/step - loss: 1.4765 - accuracy: 0.5527 - val_loss: 1.4935 - val_accuracy: 0.5358
Epoch 2/30
313/313 [==============================] - 1s 3ms/step - loss: 1.4740 - accuracy: 0.5561 - val_loss: 1.4866 - val_accuracy: 0.5456
Epoch 3/30
313/313 [==============================] - 1s 3ms/step - loss: 1.4718 - accuracy: 0.5615 - val_loss: 1.4942 - val_accuracy: 0.5248
Epoch 4/30
313/313 [==============================] - 1s 3ms/step - loss: 1.4687 - accuracy: 0.5631 - val_loss: 1.4918 - val_accuracy: 0.5282
Epoch 5/30
313/313 [==============================] - 1s 3ms/step - loss: 1.4662 - accuracy: 0.5642 - val_loss: 1.4935 - val_accuracy: 0.5216
Epoch 6/30
313/313 [==============================] - 1s 3ms/step - loss: 1.4647 - accuracy: 0.5675 - val_loss: 1.4908 - val_accuracy: 0.5332
Epoch 7/30
313/313 [==============================] - 1s 3ms/step - loss: 1.4613 - accuracy: 0.5701 - val_loss: 1.4926 - val_accuracy: 0.5380
Epoch 

In [78]:
testdf = pd.read_csv('test.csv',index_col = False); del testdf['id']

In [80]:
testdf[['C1','C2','C3','C4','C5']] = testdf[['C1','C2','C3','C4','C5']].astype('float64')
testdf[['S1','S2','S3','S4','S5']] = testdf[['S1','S2','S3','S4','S5']].astype('str')

In [87]:
hartest = testdf.values

In [85]:
testdf['ycol'] = 0

In [88]:
X_test,_ = prepare_data(hartest,means)

(1000000, 25)

In [90]:
X_test[0]

array([1.        , 0.        , 0.        , 0.        , 0.71477156,
       0.        , 1.        , 0.        , 0.        , 0.14256805,
       0.        , 0.        , 1.        , 0.        , 0.21385329,
       0.        , 0.        , 1.        , 0.        , 0.57616439,
       1.        , 0.        , 0.        , 0.        , 0.07181086])

In [107]:
hands = model.predict_classes(X_test)

In [108]:
hands = pd.Series(hands)

In [109]:
hands.value_counts()

0    549271
1    423383
2     16914
3      8155
5      1944
4       286
6        20
9        14
8         7
7         6
dtype: int64

In [111]:
submission = pd.read_csv('sampleSubmission.csv', index_col = False)

In [113]:
submission.hand = hands

In [115]:
submission.to_csv('baselineMLP.csv',index=False)